# Custom Dot Detection by Threshold Optimization

In [2]:
#custom function
from daostarfinder_dotdetection import *
#enhance figure display
%config InlineBackend.figure_format = 'retina'

ModuleNotFoundError: No module named 'daostarfinder_dotdetection'

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_2d_locs_on_2d_image(df_locs_2d_1, df_locs_2d_2, img_2d,add_trace = True, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d_1.x,
        y=df_locs_2d_1.y,
        mode='markers',
        marker_symbol='cross',
        marker=dict(
            #maxdisplayed=1000,
            size=4
            ),
        name = "Gaussian"
        )
    )
    if add_trace == True:
        fig.add_trace(go.Scattergl(
            x=df_locs_2d_2.x,
            y=df_locs_2d_2.y,
            mode='markers',
            marker_symbol='cross',
            marker=dict(
                #maxdisplayed=1000,
                size=4
                ),
            name = "LoG"
            )
        )
    #-------------------------------------------
    
    fig.show()
    
def plot_3d_locs_on_2d_image(df_tiff_1, df_tiff_2, tiff, channel, 
                             raw_src = None, raw_image = False, add_trace = True, zmax=10):
    
    if raw_image == False:

        #PLot All Z's that had dot detection
        #-------------------------------------------
        for z in range(len(tiff[:,channel])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1, None, tiff[z, channel], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel],add_trace=add_trace, zmax=zmax)
    else:
        #read raw image
        tiff = tf.imread(raw_src)
        print("shape =", tiff.shape)
        #plot
        for z in range(len(tiff[:,channel])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,None, tiff[z, channel], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel],add_trace=add_trace, zmax=zmax)

In [ ]:
import time
#start time
start = time.time()

#image sources
img_src = "/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/notebook_pyfiles/pre_processed_images/HybCycle_0/MMStack_Pos1.ome.tif"
img_raw = '/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/notebook_pyfiles/aberration_corrected/HybCycle_0/MMStack_Pos1.ome.tif'
raw_src=None

#img_src: path to image
#box_size: size of bounding box for peak local max detection
#exclude_border: number of pixels to exclude from the border
#channel: which channel you want to analyze
#pos: position number if check initial is True
#choose_thresh_set:which threshold set you want to use (if you have already ran full screen)
#hyb_number:total number of hybs for choose thresh set(if you are picking one)
#check_initial: check initial optimal threshold
#region_size: bounding box size for gaussian fit or radial centering
#gaussian: bool for performing gaussian fit
#radial_center: bool for radial centering
#optimize: bool to test different threshold and min dots
#output: bool to write out results or return data

dots = dot_detection(img_src, box_size = 5, exclude_border=3, HybCycle=0, size_cutoff=3, 
                     opt_thresh=200,channel=1,pos=1,choose_thresh_set = 0, hyb_number=12,
                     check_initial = False, region_size=7,
                     gaussian = True, radial_center=False, optimize=True, output=False)
print(f"This task took {(time.time() - start)/60} minutes")

In [ ]:
dots

In [ ]:
# #from dotdetection opt
# thresh = 4
# dots = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dots_comb/Channel_1_radialcenter/MMStack_Pos0/Threshold_{thresh}/Dot_Locations/locations_z_0.csv")

In [ ]:
# #isolate hyb
# hyb = 0
# check = dots[dots["hyb"]==hyb]

In [ ]:
img_raw = '/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/notebook_pyfiles/aberration_corrected/HybCycle_0/MMStack_Pos1.ome.tif'
plot_3d_locs_on_2d_image(df,None, tiff=None, channel=0, raw_src = img_raw, 
                         raw_image = True, add_trace=False, zmax=1000)

In [ ]:
plot_3d_locs_on_2d_image(dots,None, tiff=None, channel=0, raw_src = img_raw, 
                         raw_image = True, add_trace=False, zmax=1000)

In [ ]:
tiff = tf.imread(img_src)
plot_3d_locs_on_2d_image(df,None, tiff=tiff, channel=0, 
                         raw_src = img_raw, raw_image = False,add_trace=False, zmax=1000)

# Organize dots detected files

In [ ]:
#custom functions
from dot_detection_peaklocal_rad_gaus import combine_dot_files

In [ ]:
path_dots = "/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/notebook_pyfiles/dots_detected/Channel_4"
combine_dot_files(path_dots, hyb_start=0,hyb_end=11,num_HybCycle = 12,
                  pos = 1, channel=4, num_z = 2, opt_files = True)

# Check if there are weird dropouts in a certain hyb for each threshold

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
#collect arrays
channel = 4
comb_dots_list = []
for i in range(11):
    path = f"/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/notebook_pyfiles/dots_comb/Channel_{channel}/MMStack_Pos1/Threshold_{i}/Dot_Locations/locations_z_0.csv"
    arr1  = pd.read_csv(path)
    comb_dots_list.append(arr1)

In [ ]:
final_counts = []
for df in comb_dots_list:
    dots_per_hyb = []
    for i in df["hyb"].unique():
        dots_per_hyb.append(len(df[df["hyb"]==i]))
    final_counts.append(dots_per_hyb)

In [ ]:
#generate subplot canvas
x = 4
fig, ax = plt.subplots(x, x, figsize=(8,10), sharex = True, constrained_layout=True)
fig.text(0.5, 0.2, 'HybCycles', ha='center')
fig.text(-0.03, 0.6, 'Total Counts', va='center', rotation='vertical')
i = 0

#fill subplots
for row in ax:
    for col in row:
        try:
            #generate scatter plot of gene
            col.bar(x=np.arange(1,len(final_counts[i])+1,1) ,height=final_counts[i])
            sns.despine()
            col.set_title(f"Threshold {i}")
        except IndexError:
            col.remove()
        i += 1
plt.show()

# Combine threshold channels (for dash)

In [ ]:
channel = [1,2,3,4]
z = 1
df_list = []
for i in range(11):
    thresh_df = []
    for c in channel:
        paths = f"/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/notebook_pyfiles/dots_comb/Channel_{c}/MMStack_Pos1/Threshold_{i}/Dot_Locations/locations_z_{z}.csv"
        df = pd.read_csv(paths)
        thresh_df.append(df)
    df_comb = pd.concat(thresh_df)
    df_list.append(df_comb)

In [ ]:
from pathlib import Path

for i in range(len(df_list)):
    df = df_list[i].reset_index(drop=True)
    path = Path(f"/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/notebook_pyfiles/dots_comb/channels_combined_nodecon/Threshold_{i}")
    path.mkdir(parents=True,exist_ok=True)
    output = path / f"locations_z_{z}.csv"
    df.to_csv(str(output))

In [ ]:
import pandas as pd
df = pd.read_csv("../../dots_comb/channels_combined_nodecon/Threshold_0/locations_z_1.csv")

In [ ]:
df